# 🚀 ENTRAÎNEMENT COMPLET - Google Colab avec GPU

## Configuration Complète pour l'Entraînement du Robot de Navigation

Ce notebook permet un entraînement complet avec:
- ✅ **GPU gratuit** (T4/V100/A100)
- ✅ **Sauvegarde automatique** sur Google Drive
- ✅ **2000 épisodes** d'entraînement
- ✅ **Curriculum learning** (4 stages progressifs)
- ✅ **TensorBoard** en temps réel
- ✅ **Auto-resume** en cas de timeout
- ✅ **Visualisations** et rapport PDF
- ✅ **Test du modèle** final

---

### 📋 Instructions:
1. **Runtime** → **Change runtime type** → **GPU (T4)**
2. **Runtime** → **Run all**
3. Autorise l'accès à Google Drive
4. Attends ~30-45 minutes
5. Récupère les résultats dans Drive!

---

**Date:** December 6, 2025  
**Version:** 2.0 - Production Ready

## 🔧 Étape 1: Configuration Initiale et Vérification GPU

In [ ]:
import os
import sys
from pathlib import Path

# Vérification GPU
print("=" * 70)
print("🔍 VÉRIFICATION SYSTÈME")
print("=" * 70)

# Check if running on Colab
try:
    import google.colab
    IN_COLAB = True
    print("✅ Environnement: Google Colab")
except:
    IN_COLAB = False
    print("⚠️  Environnement: Local")

# Check GPU
import torch
if torch.cuda.is_available():
    print(f"✅ GPU Détecté: {torch.cuda.get_device_name(0)}")
    print(f"   Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    device = torch.device('cuda')
else:
    print("⚠️  GPU non disponible - Utilisation CPU")
    device = torch.device('cpu')

print(f"   Device: {device}")
print(f"   PyTorch Version: {torch.__version__}")
print("=" * 70)

## 📁 Étape 2: Montage Google Drive et Extraction du Projet

In [ ]:
if IN_COLAB:
    # Monter Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Définir les chemins
    DRIVE_ROOT = Path('/content/drive/MyDrive/RL_Project')
    PROJECT_ZIP = DRIVE_ROOT / 'projet_RL.zip'
    WORK_DIR = Path('/content/projet_RL')
    RESULTS_DIR = DRIVE_ROOT / 'results'
    CHECKPOINTS_DIR = DRIVE_ROOT / 'checkpoints'
    
    print("\n✅ Google Drive monté avec succès!")
    print(f"   Drive Root: {DRIVE_ROOT}")
    
    # Créer les dossiers de résultats
    RESULTS_DIR.mkdir(parents=True, exist_ok=True)
    CHECKPOINTS_DIR.mkdir(parents=True, exist_ok=True)
    
    # Extraire le projet
    if PROJECT_ZIP.exists():
        print(f"\n📦 Extraction du projet depuis {PROJECT_ZIP}...")
        import zipfile
        with zipfile.ZipFile(PROJECT_ZIP, 'r') as zip_ref:
            zip_ref.extractall(WORK_DIR)
        print(f"✅ Projet extrait dans: {WORK_DIR}")
    else:
        print(f"❌ ERREUR: {PROJECT_ZIP} n'existe pas!")
        print(f"   Upload 'projet_RL.zip' dans '{DRIVE_ROOT}'")
        sys.exit(1)
    
    # Changer vers le répertoire du projet
    os.chdir(WORK_DIR)
    sys.path.insert(0, str(WORK_DIR))
    
else:
    # Mode local
    WORK_DIR = Path.cwd()
    RESULTS_DIR = WORK_DIR / 'results' / 'colab_training'
    CHECKPOINTS_DIR = WORK_DIR / 'checkpoints' / 'colab'
    RESULTS_DIR.mkdir(parents=True, exist_ok=True)
    CHECKPOINTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n📂 Répertoire de travail: {WORK_DIR}")
print(f"📊 Résultats: {RESULTS_DIR}")
print(f"💾 Checkpoints: {CHECKPOINTS_DIR}")

## 📦 Étape 3: Installation des Dépendances

In [ ]:
%%capture
# Installation silencieuse des dépendances

print("📦 Installation des dépendances...")

!pip install torch torchvision torchaudio --upgrade
!pip install gymnasium numpy matplotlib seaborn pandas
!pip install tensorboard optuna tqdm rich
!pip install pygame pillow opencv-python
!pip install plotly kaleido

# Installer le projet
!pip install -e .

print("✅ Installation terminée!")

## 🎯 Étape 4: Configuration de l'Entraînement Complet

In [ ]:
import sys
import time
import json
from pathlib import Path
from datetime import datetime

# Imports PyTorch
import torch
import torch.nn as nn
import numpy as np

# Imports Gymnasium
import gymnasium as gym

# Imports pour l'environnement
from src.environment.navigation_env import NavigationEnv
from src.environment.obstacles import StaticObstacle, DynamicObstacle

# Imports pour l'agent
from src.agents.dqn_agent import DQNAgent

# Imports pour le training
from src.training.curriculum_learning import CurriculumLearningSystem
from src.training.experiment_tracker import UnifiedTracker

# Imports pour les utilitaires
from src.utils.replay_buffer import ReplayBuffer

# Imports pour la visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output, display

# Configuration matplotlib pour Colab
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✅ Tous les modules importés avec succès!")
print(f"📍 Répertoire de travail: {Path.cwd()}")

## ⚙️ Étape 5: Hyperparamètres et Configuration

In [ ]:
# ==================== CONFIGURATION DE L'ENTRAÎNEMENT ====================

CONFIG = {
    # 🎮 Paramètres d'entraînement
    'num_episodes': 2000,              # Nombre total d'épisodes
    'max_steps_per_episode': 1000,     # Nombre max de pas par épisode
    'target_update_frequency': 10,     # Fréquence de mise à jour du réseau cible
    
    # 🧠 Architecture du réseau
    'hidden_dims': [512, 512],         # Dimensions des couches cachées
    'learning_rate': 5e-4,             # Taux d'apprentissage
    'gamma': 0.99,                     # Facteur de discount
    
    # 💾 Replay Buffer
    'buffer_capacity': 100000,         # Capacité du buffer
    'batch_size': 128,                 # Taille du batch
    
    # 🎯 Epsilon (exploration)
    'epsilon_start': 1.0,              # Epsilon initial
    'epsilon_end': 0.01,               # Epsilon final
    'epsilon_decay': 0.995,            # Décroissance epsilon
    
    # 🗺️ Environnement
    'env_width': 800,                  # Largeur de l'environnement
    'env_height': 600,                 # Hauteur de l'environnement
    'num_obstacles': 7,                # Nombre d'obstacles
    
    # 💾 Sauvegarde
    'checkpoint_interval': 100,        # Sauvegarder tous les N épisodes
    'progress_interval': 25,           # Afficher progrès tous les N épisodes
    
    # ⏱️ Gestion du timeout Colab (12h max)
    'max_runtime_hours': 11.5,         # Buffer de 30 min avant timeout
    'auto_save_interval': 50,          # Sauvegarde auto tous les N épisodes
}

# Chemins pour la sauvegarde
if IN_COLAB:
    CHECKPOINT_PATH = Path(CHECKPOINTS_DIR) / "checkpoint_colab.pt"
    BEST_MODEL_PATH = Path(CHECKPOINTS_DIR) / "best_model_colab.pt"
    TENSORBOARD_DIR = Path(WORK_DIR) / "runs" / "colab_training"
else:
    CHECKPOINT_PATH = Path("checkpoints") / "checkpoint_local.pt"
    BEST_MODEL_PATH = Path("checkpoints") / "best_model_local.pt"
    TENSORBOARD_DIR = Path("runs") / "local_training"

# Créer les dossiers si nécessaire
CHECKPOINT_PATH.parent.mkdir(parents=True, exist_ok=True)
TENSORBOARD_DIR.mkdir(parents=True, exist_ok=True)

print("⚙️ Configuration:")
print(f"   📊 Épisodes: {CONFIG['num_episodes']}")
print(f"   🧠 Architecture: {CONFIG['hidden_dims']}")
print(f"   📈 Learning rate: {CONFIG['learning_rate']}")
print(f"   💾 Buffer size: {CONFIG['buffer_capacity']}")
print(f"   🎯 Batch size: {CONFIG['batch_size']}")
print(f"   🗺️  Environnement: {CONFIG['env_width']}x{CONFIG['env_height']}")
print(f"   🚧 Obstacles: {CONFIG['num_obstacles']}")
print(f"   💾 Checkpoint tous les {CONFIG['checkpoint_interval']} épisodes")
print(f"   ⏱️  Runtime max: {CONFIG['max_runtime_hours']}h")
print(f"\n📁 Chemins:")
print(f"   Checkpoint: {CHECKPOINT_PATH}")
print(f"   Meilleur modèle: {BEST_MODEL_PATH}")
print(f"   TensorBoard: {TENSORBOARD_DIR}")

## 🏗️ Étape 6: Initialisation de l'Environnement et de l'Agent

In [ ]:
# ==================== CRÉATION DES OBSTACLES ====================
print("🚧 Création des obstacles...")

obstacles = []
np.random.seed(42)  # Pour la reproductibilité

for i in range(CONFIG['num_obstacles']):
    x = np.random.randint(100, CONFIG['env_width'] - 100)
    y = np.random.randint(100, CONFIG['env_height'] - 100)
    width = np.random.randint(40, 80)
    height = np.random.randint(40, 80)
    
    obstacle = StaticObstacle(
        x=x, y=y,
        width=width, height=height,
        color=(100, 100, 100)
    )
    obstacles.append(obstacle)

print(f"✅ {len(obstacles)} obstacles créés")

# ==================== CRÉATION DE L'ENVIRONNEMENT ====================
print("\n🗺️  Création de l'environnement...")

env = NavigationEnv(
    width=CONFIG['env_width'],
    height=CONFIG['env_height'],
    obstacles=obstacles
)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

print(f"✅ Environnement créé")
print(f"   📊 Dimension état: {state_dim}")
print(f"   🎮 Nombre d'actions: {action_dim}")

# ==================== CRÉATION DE L'AGENT ====================
print("\n🤖 Création de l'agent DQN...")

agent = DQNAgent(
    state_dim=state_dim,
    action_dim=action_dim,
    hidden_dims=CONFIG['hidden_dims'],
    learning_rate=CONFIG['learning_rate'],
    gamma=CONFIG['gamma']
)

# Déplacer l'agent sur le device (GPU si disponible)
agent.q_network.to(device)
agent.target_network.to(device)

print(f"✅ Agent créé et déplacé sur {device}")
print(f"   🧠 Architecture: {CONFIG['hidden_dims']}")
print(f"   📈 Learning rate: {CONFIG['learning_rate']}")

# ==================== CRÉATION DU REPLAY BUFFER ====================
print("\n💾 Création du Replay Buffer...")

replay_buffer = ReplayBuffer(capacity=CONFIG['buffer_capacity'])

print(f"✅ Replay Buffer créé (capacité: {CONFIG['buffer_capacity']:,})")

# ==================== SYSTÈME D'APPRENTISSAGE PAR CURRICULUM ====================
print("\n📚 Initialisation du Curriculum Learning...")

curriculum = CurriculumLearningSystem(
    num_episodes=CONFIG['num_episodes'],
    stages=None  # Utilise les stages par défaut
)

print(f"✅ Curriculum Learning initialisé")
print(f"   📊 Nombre de stages: {len(curriculum.stages)}")

# ==================== TRACKER D'EXPÉRIENCES ====================
print("\n📊 Initialisation du tracker...")

tracker = UnifiedTracker(
    project_name="robot-navigation-colab",
    experiment_name=f"training_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
    config=CONFIG,
    log_dir=str(TENSORBOARD_DIR),
    use_tensorboard=True,
    use_wandb=False,  # Désactivé pour Colab
    use_mlflow=False  # Désactivé pour Colab
)

print(f"✅ Tracker initialisé (TensorBoard)")
print(f"   📁 Log dir: {TENSORBOARD_DIR}")

print("\n" + "="*60)
print("🚀 TOUT EST PRÊT POUR L'ENTRAÎNEMENT!")
print("="*60)

## 🎯 Étape 7: Boucle d'Entraînement Principale (2000 Épisodes)

In [ ]:
# ==================== FONCTIONS UTILITAIRES ====================

def save_checkpoint(episode, agent, optimizer, best_reward, epsilon, curriculum, filename):
    """Sauvegarde un checkpoint complet"""
    checkpoint = {
        'episode': episode,
        'q_network_state_dict': agent.q_network.state_dict(),
        'target_network_state_dict': agent.target_network.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_reward': best_reward,
        'epsilon': epsilon,
        'curriculum_stage': curriculum.current_stage_idx,
        'config': CONFIG
    }
    torch.save(checkpoint, filename)
    print(f"💾 Checkpoint sauvegardé: {filename}")

def load_checkpoint(filename, agent, optimizer):
    """Charge un checkpoint"""
    if not Path(filename).exists():
        return 0, float('-inf'), CONFIG['epsilon_start'], 0
    
    checkpoint = torch.load(filename, map_location=device)
    agent.q_network.load_state_dict(checkpoint['q_network_state_dict'])
    agent.target_network.load_state_dict(checkpoint['target_network_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    print(f"📂 Checkpoint chargé: {filename}")
    print(f"   Episode: {checkpoint['episode']}")
    print(f"   Best reward: {checkpoint['best_reward']:.2f}")
    
    return (checkpoint['episode'], 
            checkpoint['best_reward'], 
            checkpoint['epsilon'],
            checkpoint['curriculum_stage'])

# ==================== TENTATIVE DE CHARGEMENT D'UN CHECKPOINT ====================
print("🔍 Recherche d'un checkpoint existant...")

start_episode = 0
best_reward = float('-inf')
epsilon = CONFIG['epsilon_start']
start_stage = 0

if CHECKPOINT_PATH.exists():
    try:
        start_episode, best_reward, epsilon, start_stage = load_checkpoint(
            CHECKPOINT_PATH, agent, agent.optimizer
        )
        curriculum.current_stage_idx = start_stage
        print(f"✅ Reprise depuis l'épisode {start_episode}")
    except Exception as e:
        print(f"⚠️  Erreur lors du chargement: {e}")
        print("🔄 Démarrage d'un nouvel entraînement")
else:
    print("📝 Pas de checkpoint trouvé, démarrage d'un nouvel entraînement")

# ==================== VARIABLES DE SUIVI ====================
episode_rewards = []
episode_lengths = []
episode_losses = []
episode_successes = []
training_start_time = time.time()

print("\n" + "="*60)
print(f"🚀 DÉBUT DE L'ENTRAÎNEMENT - {CONFIG['num_episodes']} ÉPISODES")
print("="*60)

# ==================== BOUCLE D'ENTRAÎNEMENT ====================
for episode in range(start_episode, CONFIG['num_episodes']):
    # Vérification du timeout Colab
    elapsed_hours = (time.time() - training_start_time) / 3600
    if elapsed_hours >= CONFIG['max_runtime_hours']:
        print(f"\n⏱️  Timeout approché ({elapsed_hours:.2f}h)")
        print("💾 Sauvegarde automatique avant arrêt...")
        save_checkpoint(episode, agent, agent.optimizer, best_reward, 
                       epsilon, curriculum, CHECKPOINT_PATH)
        print("✅ Sauvegarde terminée. Vous pouvez relancer le notebook.")
        break
    
    # Obtenir la configuration du curriculum
    curriculum.update_stage(episode)
    stage_config = curriculum.get_current_env_config()
    current_epsilon = curriculum.get_current_epsilon(epsilon)
    
    # Reset de l'environnement
    state, _ = env.reset()
    episode_reward = 0
    episode_loss = 0
    num_updates = 0
    done = False
    steps = 0
    
    # Épisode
    while not done and steps < CONFIG['max_steps_per_episode']:
        # Sélection de l'action
        action = agent.select_action(state, current_epsilon)
        
        # Exécution de l'action
        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        
        # Stockage dans le replay buffer
        replay_buffer.add(state, action, reward, next_state, done)
        
        # Entraînement si le buffer a assez d'expériences
        if len(replay_buffer.buffer) >= CONFIG['batch_size']:
            batch = replay_buffer.sample(CONFIG['batch_size'])
            loss = agent.train_step(batch)
            episode_loss += loss
            num_updates += 1
        
        state = next_state
        episode_reward += reward
        steps += 1
    
    # Mise à jour du réseau cible
    if (episode + 1) % CONFIG['target_update_frequency'] == 0:
        agent.update_target_network()
    
    # Décroissance d'epsilon
    epsilon = max(CONFIG['epsilon_end'], epsilon * CONFIG['epsilon_decay'])
    
    # Enregistrement des métriques
    episode_rewards.append(episode_reward)
    episode_lengths.append(steps)
    avg_loss = episode_loss / max(num_updates, 1)
    episode_losses.append(avg_loss)
    success = info.get('success', False)
    episode_successes.append(1 if success else 0)
    
    # Logging vers TensorBoard
    tracker.log_metrics({
        'reward': episode_reward,
        'episode_length': steps,
        'loss': avg_loss,
        'epsilon': current_epsilon,
        'success': 1 if success else 0,
        'curriculum_stage': curriculum.current_stage_idx,
        'buffer_size': len(replay_buffer.buffer)
    }, step=episode)
    
    # Sauvegarde du meilleur modèle
    if episode_reward > best_reward:
        best_reward = episode_reward
        torch.save(agent.q_network.state_dict(), BEST_MODEL_PATH)
    
    # Sauvegarde périodique
    if (episode + 1) % CONFIG['checkpoint_interval'] == 0:
        save_checkpoint(episode + 1, agent, agent.optimizer, best_reward,
                       epsilon, curriculum, CHECKPOINT_PATH)
    
    # Affichage des progrès
    if (episode + 1) % CONFIG['progress_interval'] == 0:
        clear_output(wait=True)
        
        avg_reward_100 = np.mean(episode_rewards[-100:])
        avg_length_100 = np.mean(episode_lengths[-100:])
        success_rate_100 = np.mean(episode_successes[-100:]) * 100
        
        print(f"{'='*60}")
        print(f"📊 Épisode {episode + 1}/{CONFIG['num_episodes']}")
        print(f"{'='*60}")
        print(f"   💰 Reward: {episode_reward:.2f}")
        print(f"   📈 Moyenne 100 derniers: {avg_reward_100:.2f}")
        print(f"   🎯 Meilleur reward: {best_reward:.2f}")
        print(f"   📏 Longueur: {steps} pas")
        print(f"   ✅ Succès: {'Oui' if success else 'Non'}")
        print(f"   📊 Taux de succès (100): {success_rate_100:.1f}%")
        print(f"   📉 Loss moyenne: {avg_loss:.4f}")
        print(f"   🎲 Epsilon: {current_epsilon:.4f}")
        print(f"   📚 Stage: {curriculum.get_current_stage_name()}")
        print(f"   💾 Buffer: {len(replay_buffer.buffer):,}/{CONFIG['buffer_capacity']:,}")
        print(f"   ⏱️  Temps: {elapsed_hours:.2f}h / {CONFIG['max_runtime_hours']}h")
        print(f"{'='*60}")
        
        # Mini graphique de progression
        if len(episode_rewards) >= 100:
            plt.figure(figsize=(12, 4))
            
            plt.subplot(1, 3, 1)
            plt.plot(episode_rewards[-100:])
            plt.title('Rewards (100 derniers)')
            plt.xlabel('Épisode')
            plt.ylabel('Reward')
            plt.grid(True)
            
            plt.subplot(1, 3, 2)
            plt.plot(episode_losses[-100:])
            plt.title('Loss (100 derniers)')
            plt.xlabel('Épisode')
            plt.ylabel('Loss')
            plt.grid(True)
            
            plt.subplot(1, 3, 3)
            success_window = np.convolve(episode_successes, np.ones(10)/10, mode='valid')
            plt.plot(success_window[-90:] * 100)
            plt.title('Taux de succès (fenêtre mobile 10)')
            plt.xlabel('Épisode')
            plt.ylabel('Succès (%)')
            plt.grid(True)
            
            plt.tight_layout()
            plt.show()

# Sauvegarde finale
print(f"\n{'='*60}")
print("🏁 ENTRAÎNEMENT TERMINÉ!")
print(f"{'='*60}")
save_checkpoint(CONFIG['num_episodes'], agent, agent.optimizer, best_reward,
               epsilon, curriculum, CHECKPOINT_PATH)

tracker.finish()

print(f"\n✅ Meilleur reward obtenu: {best_reward:.2f}")
print(f"✅ Taux de succès final: {np.mean(episode_successes[-100:]) * 100:.1f}%")
print(f"✅ Durée totale: {(time.time() - training_start_time) / 3600:.2f}h")

## 📊 Étape 8: Visualisation des Résultats d'Entraînement

In [ ]:
# ==================== VISUALISATION COMPLÈTE DES RÉSULTATS ====================

print("📊 Génération des visualisations...")

fig = plt.figure(figsize=(16, 12))

# 1. Courbe des rewards
ax1 = plt.subplot(3, 2, 1)
plt.plot(episode_rewards, alpha=0.6, label='Reward par épisode')
# Moyenne mobile sur 50 épisodes
if len(episode_rewards) >= 50:
    ma_50 = np.convolve(episode_rewards, np.ones(50)/50, mode='valid')
    plt.plot(range(49, len(episode_rewards)), ma_50, 'r-', linewidth=2, label='Moyenne mobile 50')
plt.xlabel('Épisode')
plt.ylabel('Reward')
plt.title('Évolution des Rewards')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. Courbe des losses
ax2 = plt.subplot(3, 2, 2)
plt.plot(episode_losses, alpha=0.6, label='Loss par épisode')
if len(episode_losses) >= 50:
    ma_loss_50 = np.convolve(episode_losses, np.ones(50)/50, mode='valid')
    plt.plot(range(49, len(episode_losses)), ma_loss_50, 'r-', linewidth=2, label='Moyenne mobile 50')
plt.xlabel('Épisode')
plt.ylabel('Loss')
plt.title('Évolution de la Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. Taux de succès
ax3 = plt.subplot(3, 2, 3)
if len(episode_successes) >= 50:
    success_ma = np.convolve(episode_successes, np.ones(50)/50, mode='valid') * 100
    plt.plot(range(49, len(episode_successes)), success_ma, 'g-', linewidth=2)
plt.xlabel('Épisode')
plt.ylabel('Taux de succès (%)')
plt.title('Taux de Succès (Moyenne mobile 50)')
plt.grid(True, alpha=0.3)

# 4. Longueur des épisodes
ax4 = plt.subplot(3, 2, 4)
plt.plot(episode_lengths, alpha=0.6)
if len(episode_lengths) >= 50:
    ma_len_50 = np.convolve(episode_lengths, np.ones(50)/50, mode='valid')
    plt.plot(range(49, len(episode_lengths)), ma_len_50, 'r-', linewidth=2)
plt.xlabel('Épisode')
plt.ylabel('Nombre de pas')
plt.title('Longueur des Épisodes')
plt.grid(True, alpha=0.3)

# 5. Distribution des rewards (100 derniers)
ax5 = plt.subplot(3, 2, 5)
plt.hist(episode_rewards[-100:], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.xlabel('Reward')
plt.ylabel('Fréquence')
plt.title('Distribution des Rewards (100 derniers épisodes)')
plt.grid(True, alpha=0.3)

# 6. Statistiques finales
ax6 = plt.subplot(3, 2, 6)
ax6.axis('off')
stats_text = f"""
STATISTIQUES FINALES

Nombre d'épisodes: {len(episode_rewards)}
Meilleur reward: {best_reward:.2f}

Reward moyen (tout): {np.mean(episode_rewards):.2f}
Reward moyen (100 derniers): {np.mean(episode_rewards[-100:]):.2f}

Taux succès (tout): {np.mean(episode_successes)*100:.1f}%
Taux succès (100 derniers): {np.mean(episode_successes[-100:])*100:.1f}%

Loss finale: {episode_losses[-1]:.4f}
Longueur moyenne: {np.mean(episode_lengths):.0f} pas

Buffer final: {len(replay_buffer.buffer):,}
Epsilon final: {epsilon:.4f}
Stage final: {curriculum.get_current_stage_name()}
"""
ax6.text(0.1, 0.5, stats_text, fontsize=12, family='monospace',
         verticalalignment='center')

plt.tight_layout()
plt.savefig(Path(RESULTS_DIR) / 'training_results.png', dpi=150, bbox_inches='tight')
print(f"✅ Graphique sauvegardé: {Path(RESULTS_DIR) / 'training_results.png'}")
plt.show()

# ==================== SAUVEGARDE DES DONNÉES ====================
print("\n💾 Sauvegarde des données...")

# Sauvegarde CSV
import pandas as pd

df_results = pd.DataFrame({
    'episode': range(len(episode_rewards)),
    'reward': episode_rewards,
    'length': episode_lengths,
    'loss': episode_losses,
    'success': episode_successes
})

csv_path = Path(RESULTS_DIR) / 'training_history.csv'
df_results.to_csv(csv_path, index=False)
print(f"✅ Historique sauvegardé: {csv_path}")

# Sauvegarde JSON avec statistiques
results_summary = {
    'config': CONFIG,
    'training_info': {
        'num_episodes': len(episode_rewards),
        'best_reward': float(best_reward),
        'final_epsilon': float(epsilon),
        'final_stage': curriculum.get_current_stage_name(),
        'training_duration_hours': (time.time() - training_start_time) / 3600
    },
    'statistics': {
        'mean_reward_all': float(np.mean(episode_rewards)),
        'mean_reward_last_100': float(np.mean(episode_rewards[-100:])),
        'std_reward_last_100': float(np.std(episode_rewards[-100:])),
        'success_rate_all': float(np.mean(episode_successes)),
        'success_rate_last_100': float(np.mean(episode_successes[-100:])),
        'mean_episode_length': float(np.mean(episode_lengths)),
        'final_buffer_size': len(replay_buffer.buffer)
    }
}

json_path = Path(RESULTS_DIR) / 'training_summary.json'
with open(json_path, 'w') as f:
    json.dump(results_summary, f, indent=2)
print(f"✅ Résumé sauvegardé: {json_path}")

print("\n" + "="*60)
print("✅ TOUTES LES VISUALISATIONS ET DONNÉES SAUVEGARDÉES!")
print("="*60)

## 🧪 Étape 9: Évaluation du Modèle Entraîné

In [ ]:
# ==================== ÉVALUATION SUR 50 ÉPISODES DE TEST ====================

print("🧪 Évaluation du meilleur modèle...")
print("="*60)

# Charger le meilleur modèle
if BEST_MODEL_PATH.exists():
    agent.q_network.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device))
    print(f"✅ Meilleur modèle chargé: {BEST_MODEL_PATH}")
else:
    print("⚠️  Utilisation du modèle actuel (pas de meilleur modèle sauvegardé)")

agent.q_network.eval()  # Mode évaluation

num_test_episodes = 50
test_rewards = []
test_lengths = []
test_successes = []

print(f"\n🎮 Lancement de {num_test_episodes} épisodes de test...")
print("(Sans exploration - epsilon=0)\n")

for test_ep in range(num_test_episodes):
    state, _ = env.reset()
    episode_reward = 0
    steps = 0
    done = False
    
    while not done and steps < CONFIG['max_steps_per_episode']:
        # Action sans exploration (epsilon=0)
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            q_values = agent.q_network(state_tensor)
            action = q_values.argmax(dim=1).item()
        
        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        
        state = next_state
        episode_reward += reward
        steps += 1
    
    test_rewards.append(episode_reward)
    test_lengths.append(steps)
    success = info.get('success', False)
    test_successes.append(1 if success else 0)
    
    if (test_ep + 1) % 10 == 0:
        print(f"Episode {test_ep + 1}/{num_test_episodes}: "
              f"Reward={episode_reward:.2f}, "
              f"Longueur={steps}, "
              f"Succès={'✅' if success else '❌'}")

# ==================== STATISTIQUES D'ÉVALUATION ====================
print("\n" + "="*60)
print("📊 RÉSULTATS DE L'ÉVALUATION")
print("="*60)
print(f"Reward moyen: {np.mean(test_rewards):.2f} ± {np.std(test_rewards):.2f}")
print(f"Reward min: {np.min(test_rewards):.2f}")
print(f"Reward max: {np.max(test_rewards):.2f}")
print(f"Longueur moyenne: {np.mean(test_lengths):.1f} pas")
print(f"Taux de succès: {np.mean(test_successes)*100:.1f}% ({sum(test_successes)}/{num_test_episodes})")
print("="*60)

# ==================== VISUALISATION DES RÉSULTATS DE TEST ====================
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Rewards de test
axes[0].bar(range(num_test_episodes), test_rewards, alpha=0.7, color='skyblue')
axes[0].axhline(y=np.mean(test_rewards), color='r', linestyle='--', label='Moyenne')
axes[0].set_xlabel('Épisode de test')
axes[0].set_ylabel('Reward')
axes[0].set_title('Rewards de Test')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Distribution des rewards
axes[1].hist(test_rewards, bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1].axvline(x=np.mean(test_rewards), color='r', linestyle='--', linewidth=2, label='Moyenne')
axes[1].set_xlabel('Reward')
axes[1].set_ylabel('Fréquence')
axes[1].set_title('Distribution des Rewards de Test')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Succès/Échecs
success_count = sum(test_successes)
fail_count = num_test_episodes - success_count
axes[2].pie([success_count, fail_count], 
           labels=['Succès', 'Échecs'],
           autopct='%1.1f%%',
           colors=['lightgreen', 'lightcoral'],
           startangle=90)
axes[2].set_title('Taux de Succès')

plt.tight_layout()
plt.savefig(Path(RESULTS_DIR) / 'evaluation_results.png', dpi=150, bbox_inches='tight')
print(f"\n✅ Visualisation sauvegardée: {Path(RESULTS_DIR) / 'evaluation_results.png'}")
plt.show()

# Sauvegarde des résultats de test
test_results = {
    'num_test_episodes': num_test_episodes,
    'mean_reward': float(np.mean(test_rewards)),
    'std_reward': float(np.std(test_rewards)),
    'min_reward': float(np.min(test_rewards)),
    'max_reward': float(np.max(test_rewards)),
    'mean_length': float(np.mean(test_lengths)),
    'success_rate': float(np.mean(test_successes)),
    'num_successes': int(sum(test_successes))
}

test_results_path = Path(RESULTS_DIR) / 'evaluation_results.json'
with open(test_results_path, 'w') as f:
    json.dump(test_results, f, indent=2)
print(f"✅ Résultats de test sauvegardés: {test_results_path}")

agent.q_network.train()  # Retour en mode entraînement

## 📈 Étape 10: Lancement de TensorBoard (Optionnel)

In [ ]:
# ==================== LANCEMENT DE TENSORBOARD ====================

if IN_COLAB:
    print("📈 Lancement de TensorBoard dans Colab...")
    
    # Charger l'extension TensorBoard
    %load_ext tensorboard
    
    # Lancer TensorBoard
    %tensorboard --logdir {TENSORBOARD_DIR}
    
    print(f"✅ TensorBoard lancé!")
    print(f"📁 Log directory: {TENSORBOARD_DIR}")
    print("\nℹ️  TensorBoard s'affiche ci-dessus.")
    print("    Vous pouvez explorer les métriques, les graphiques et les scalaires.")
else:
    print("💻 Mode local détecté")
    print(f"Pour lancer TensorBoard, exécutez dans un terminal:")
    print(f"   tensorboard --logdir={TENSORBOARD_DIR}")
    print(f"Puis ouvrez: http://localhost:6006")

## 📋 Résumé Final et Prochaines Étapes

### ✅ Ce qui a été accompli:
1. ✅ Configuration GPU et montage Google Drive
2. ✅ Installation des dépendances
3. ✅ Entraînement complet de 2000 épisodes avec curriculum learning
4. ✅ Sauvegarde automatique des checkpoints tous les 100 épisodes
5. ✅ Génération des visualisations (courbes, statistiques)
6. ✅ Évaluation du modèle sur 50 épisodes de test
7. ✅ Export des résultats (CSV, JSON, graphiques)

### 📁 Fichiers générés:
- **Checkpoints:**
  - `checkpoint_colab.pt` - Dernier checkpoint (peut reprendre l'entraînement)
  - `best_model_colab.pt` - Meilleur modèle basé sur le reward
  
- **Résultats:**
  - `training_history.csv` - Historique complet de l'entraînement
  - `training_summary.json` - Statistiques résumées
  - `evaluation_results.json` - Résultats des tests
  - `training_results.png` - Graphiques d'entraînement
  - `evaluation_results.png` - Graphiques d'évaluation

### 🚀 Prochaines étapes possibles:

#### 1. **Améliorer les performances:**
   - Augmenter le nombre d'épisodes (3000-5000)
   - Ajuster les hyperparamètres (learning rate, architecture)
   - Modifier la fonction de récompense
   - Ajouter plus d'obstacles ou changer leur configuration

#### 2. **Expérimenter avec d'autres algorithmes:**
   - Essayer PPO, A3C, ou SAC
   - Implémenter le Double DQN ou Dueling DQN
   - Tester Rainbow DQN

#### 3. **Analyser en détail:**
   - Visualiser les trajectoires de l'agent
   - Analyser les Q-values
   - Créer des heatmaps de la politique apprise
   - Générer des vidéos de l'agent en action

#### 4. **Déployer le modèle:**
   - Créer une interface interactive
   - Déployer sur un serveur web
   - Optimiser pour l'inférence (quantization, pruning)

### 💡 Pour relancer l'entraînement:
Si l'entraînement s'interrompt (timeout Colab), **il suffit de relancer toutes les cellules**. Le système détectera automatiquement le checkpoint et reprendra là où il s'était arrêté!

### 📊 Pour accéder aux résultats:
Les résultats sont automatiquement sauvegardés dans votre Google Drive:
- `/content/drive/MyDrive/RL_Project/results/`
- `/content/drive/MyDrive/RL_Project/checkpoints/`

---

**🎉 FÉLICITATIONS! Vous avez entraîné avec succès un agent DQN sur Google Colab! 🎉**